In [0]:
use workspace.mavenfuzzyfactory;

-- step 1: 
create temporary table sessions_w_min_pv_id_and_view_count
select 
  ws.website_session_id,
  min(wp.website_pageview_id) as first_pageview_id,
  count(wp.website_pageview_id) as count_pageviews
from website_sessions ws
left join website_pageviews wp
on 
  ws.website_session_id = wp.website_session_id
where 
  ws.created_at > '2012-06-01'
  and ws.created_at < '2012-08-31'
  and ws.utm_source = 'gsearch'
  and ws.utm_campaign = 'nonbrand'
group by 1

-- step 2: 
create temporary table sessions_w_counts_lander_and_created_at
select 
  swmpiavc.website_session_id,
  swmpiavc.count_pageviews,
  swmpiavc.first_pageview_id,
  wp.pageview_url as landing_page,
  wp.created_at as session_created_at
from sessions_w_min_pv_id_and_view_count as swmpiavc
left join website_pageviews as wp
on 
  swmpiavc.first_pageview_id = wp.website_pageview_id

-- step 3: 
select 
  CONCAT(YEAR(swclaca.session_created_at), WEEKOFYEAR(swclaca.session_created_at)) AS year_week,
  min(date(swclaca.session_created_at)) as week_start_date,
  -- count(distinct swclaca.website_session_id) as total_sessions,
  -- count(distinct case when swclaca.count_pageviews = 1 then swclaca.website_session_id else null end) as bounced_sessions,
  round((count(distinct case when swclaca.count_pageviews = 1 then swclaca.website_session_id else null end) * 1.0 / count(distinct swclaca.website_session_id)*100),2) as bounce_rate,
  count(distinct case when swclaca.landing_page = '/lander-1' then swclaca.website_session_id else null end) as lander_sessions,
  count(distinct case when swclaca.landing_page = '/home' then swclaca.website_session_id else null end) as home_sessions
from sessions_w_counts_lander_and_created_at as swclaca
group by 1
order by 2


